In [1]:
import json
import pandas as pd
import statistics

spiList = [5.55, 5.72, 7.3, 7.75, 8.4, 9, 8.8, 8.2]
print(statistics.mean(spiList))

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import operator

from tqdm import tqdm

7.59


In [12]:
from gensim.models.keyedvectors import FastTextKeyedVectors
fasttext = FastTextKeyedVectors.load("D:/fasttext_word2vec/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model")


C:\Users\Asus\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [27]:
with open ("lyashevskaya_freq_dict.json" , "r", encoding="utf-8") as f:
    lyashevskaya_freq_dict = json.load(f)
    
with open ("C:\Autotutor\improved_approach\colloc\music_unigr_freq.json", "r", encoding='utf-8') as f:
    unigramm_db = json.load(f)
    
    
with open ("D:\input\music_smart_colloc_freq.json", "r", encoding='utf-8') as f:
    colloc_db = json.load(f)
    
words_indexed_dict = []
with open("big_musiciancolloc_lit.txt", "r", encoding = "utf-8") as f:
    for line in f.readlines():
        words_indexed_dict.append(line[:-1])

In [55]:
words_indexed_dict[0]

'оказываться'

In [4]:
USER_NAME = 'big_musician'

In [5]:

marked_word_db = pd.read_csv(USER_NAME +  "_word_db.csv",header = None)

In [21]:
def get_vector(ngramm):
    ngramm_lemmas_list = ngramm.split()
    freq_list = []
    local_corpora_freq_list = []
    w2v_list = []
    for lemma in ngramm_lemmas_list:
        
        try:
            w2v = fasttext[lemma]
            w2v_list.append(w2v)
        except:
            pass
        """
        print(lemma)
        w2v = fasttext[lemma]
        w2v_list.append(w2v)
        """
        
        if lemma in lyashevskaya_freq_dict:
            freq_list.append(lyashevskaya_freq_dict[lemma])
        else:
            freq_list.append(0)
            
        if lemma in unigramm_db:
            local_corpora_freq_list.append(unigramm_db[lemma])
        else:
            local_corpora_freq_list.append(0)
    colleted_w2v_count = len(w2v_list)
    if colleted_w2v_count > 0:
        vect_sum = 300 * [0]
        for w2v in w2v_list:
            vect_sum += w2v
        vect_sum /=  colleted_w2v_count
    else:
        print(ngramm_lemmas_list, "none in fasttext")
        vect_sum = None
    if len(local_corpora_freq_list) > 0:
        local_corpora_mean = statistics.mean(local_corpora_freq_list)
    else:
        local_corpora_mean = None
    if len(freq_list) > 0:
        freq_mean = statistics.mean(freq_list)
    else:
        freq_mean = None
    
    #return (local_corpora_mean,freq_mean, vect_sum)#????
    return (freq_mean,local_corpora_mean, vect_sum)

In [28]:
list_of_all_words_similarity = {}
with open("D:\input\music_smart_colloc_freq.json", 'r', encoding = 'utf-8') as f:
    music_colloc_db = json.load(f)
    all_colloc = []
    all_colloc.extend(list(music_colloc_db[str(2)].keys()))
    all_colloc.extend(list(music_colloc_db[str(3)].keys()))
    all_colloc.extend(list(music_colloc_db[str(4)].keys()))
    for colloc_word_ind in tqdm(range(len(all_colloc))):
        if not all_colloc[colloc_word_ind].isdigit():
            current_word_similarity = []
            colloc_word_vector = get_vector(all_colloc[colloc_word_ind])
            if np.any(colloc_word_vector[0]) and np.any(colloc_word_vector[1]) and np.any(colloc_word_vector[2]):
                freq_vec = np.array([colloc_word_vector[0], colloc_word_vector[1]]).reshape(1,-1)
                colloc_w2v = np.array(colloc_word_vector[2]).reshape(1,-1)
                absolute_equality_found = False
                for db_word_index in range(len(marked_word_db)):
                    compared_raw_word_vect = marked_word_db.iloc[db_word_index][:-1]
                    comp_word2vec = compared_raw_word_vect[2:]
                    comp_colloc_w2v = np.array(comp_word2vec).reshape(1,-1)
                    if np.any(comp_word2vec) and np.any(colloc_w2v):
                        w2v_sim = cosine_similarity(colloc_w2v,comp_colloc_w2v)
                    else:
                        w2v_sim = [[0]]
                    comp_freq_vec = np.array([compared_raw_word_vect[1], compared_raw_word_vect[0]]).reshape(1,-1)
                    sim_freq = cosine_similarity(freq_vec,comp_freq_vec)
                    #print("freq", sim_freq)
                    #print(comp_freq_vec)
                    w2v_sim = w2v_sim[0][0]
                    sim_freq = sim_freq[0][0]

                    average_similarity = w2v_sim*0.8 + sim_freq*0.2
                    if average_similarity >= 1:
                        absolute_equality_found = True

                    word_comp_json = {"marked_colloc_db_index":db_word_index, "sim_colloc":words_indexed_dict[db_word_index],"w2v_sim":w2v_sim,"freqence_similarity":sim_freq}
                    if average_similarity> 0.6 and w2v_sim > 0.65 and absolute_equality_found == False:
                        current_word_similarity.append((word_comp_json,average_similarity))
                    elif average_similarity >= 0.75 and absolute_equality_found == True:
                        current_word_similarity.append((word_comp_json,average_similarity))
                current_word_similarity.sort(key = operator.itemgetter(1), reverse= True)
                if absolute_equality_found == True:
                    only_absolute_equalitie = []
                    for simil_el in current_word_similarity:
                        if simil_el[1] >= 0.85:
                            only_absolute_equalitie.append(simil_el)
                    list_of_all_words_similarity[all_colloc[colloc_word_ind]] = only_absolute_equalitie[:5]
                else:
                    list_of_all_words_similarity[all_colloc[colloc_word_ind]] = current_word_similarity[:5]
            
       


  2%|█▏                                                                          | 924/57842 [07:10<7:22:05,  2.15it/s]

['5', '0'] none in fasttext


  2%|█▎                                                                         | 1027/57842 [08:16<7:37:24,  2.07it/s]

[] none in fasttext


  3%|██▎                                                                        | 1817/57842 [17:24<8:56:37,  1.74it/s]

...

 61%|███████████████████████████████████████████▊                            | 35175/57842 [6:42:13<4:19:11,  1.46it/s]

[] none in fasttext


 73%|████████████████████████████████████████████████████▍                   | 42119/57842 [7:54:43<2:57:12,  1.48it/s]

['n', 'w', 'a'] none in fasttext


100%|█████████████████████████████████████████████████████████████████████████| 57842/57842 [10:38:47<00:00,  1.51it/s]


In [29]:
with open ("big_musician_ngr_colloc_similairities.json", 'w', encoding = 'utf-8') as f:
    json.dump(list_of_all_words_similarity, f , indent = 4, ensure_ascii=False)

In [30]:
with open ("big_musician_unigr_colloc_similairities.json", 'r') as f:
    data = json.load(f)

In [33]:
type(list_of_all_words_similarity),type(data)

(dict, dict)

In [35]:
d = list_of_all_words_similarity.copy()
d.update(data)

In [37]:
with open ("big_musician_overall_colloc_similairities.json", 'w', encoding = 'utf-8') as f:
    json.dump(d, f , indent = 4, ensure_ascii=False)